# Modelling metabolism in soy mature leaves

### Generating SOY core model

In [1]:
from cobra import io

#import sbml file
model = io.sbml.create_cobra_model_from_sbml_file("/home/sanu/Documents/Scripts/git/plantcoremetabolism-model/PlantCoreMetabolism_v1_2_3.xml")

In [2]:
import pythoncyc
from Functions import *

soy = pythoncyc.select_organism("soy")

In [3]:
model = addGPR2Models(model,soy)

--------------
This list of metabolic reactions are ignored
['RXN_9650_p', '2_KETO_ADIPATE_DEHYDROG_RXN_m', 'Phytol_biosynthesis_p', 'CYSTEINE_AMINOTRANSFERASE_RXN_m', 'GLYCINE_TRNA_LIGASE_RXN_c', 'RXN66_1_c', 'RXN_9648_p', 'RXN-9651', 'Plastidial_ATP_Synthase_p', 'GGPP_biosynthesis_p', 'RXN_9653_p', 'lycopene_biosynthesis_p', 'RXN_2141_p', 'SUCCINYL_COA_HYDROLASE_RXN_m', 'PROTON_ATPase_c', 'MDA_Fd_Ascorbate_p', 'MercaptoPyruvateSulfurtransferase_m', 'Phytol_degradation_p', 'RXN_9652_p', 'A_B_oxidation_x', 'unlProtHYPO_c', 'Mitochondrial_ATP_Synthase_m', 'IPP_biosynthesis_c', 'Mehler_Reaction_p', 'Beta_Oxidation_x', 'HMBPP_synthesis_p', 'OROTATE_REDUCTASE_NADH_RXN_p', 'Ferredoxin_Plastoquinone_Reductase_p', 'RXN_9651_p', 'NADPH_Dehydrogenase_p', 'Plastoquinol_Oxidase_p', 'SUCCINATE_COA_LIGASE_GDP_FORMING_RXN_m', 'RXN_1781_v', 'PREPHENATE_DEHYDROGENASE_NADP_RXN_p', 'PREPHENATEDEHYDROG_RXN_p', 'MALEYLACETOACETATE_ISOMERASE_RXN_c', 'RXN_9654_p', 'LCYSDESULF_RXN_c', 'RXN_9958_NAD_m', 'HEXO

### Importing transcript data

In [11]:
import pandas as pd
df = pd.read_csv("/home/sanu/Downloads/gmax_leaf_CO2_normExprn_1.txt",sep="\t")

In [51]:
modelGenes = dict()
for gene in model.genes:
    geneID = gene.id.split(".")[0]+"."+gene.id.split(".")[1]
    modelGenes[geneID]=gene


In [59]:
modelGenesNotInTranscript = set(modelGenes.keys()) - set(df["Glyma2.0"])

In [60]:
len(modelGenesNotInTranscript)

991

In [61]:
len(model.genes)

1899

In [62]:
i=0
for rxn in model.reactions:
    if len(rxn.genes)!=0:
        i=i+1
print i

377
